In [1]:
# IndexFlatL2 measures the L2 distance between all given points between out query vector

import requests
from io import StringIO
import pandas as pd

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [4]:
sentence_b = data["sentence_B"].tolist()
sentences.extend(sentence_b) # merge sentence_a and sentence_b
print(len(set(sentences)))

4802


In [5]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

print(f"Length: {len(urls)}, {type(urls)}")

Length: 7, <class 'list'>


In [6]:
for url in urls:
    res = requests.get(url)
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    print(data.shape)
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

(723, 3)
(701, 3)
(750, 3)
(561, 3)
(750, 3)
(750, 3)
(1500, 3)


In [7]:
print(len(sentences))

20470


In [ ]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [13]:
print(len(sentences))

14504


In [1]:
a = "his thought process was on so many levels that he gave himself a phobia of heights".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()

if type(b) == set:
    print(True)


print(type(a))

<class 'list'>


In [ ]:
def jac(x: set, y: set):
    # if not type(x) == set or not type(y) == set:
    #     raise ValueError("input must be set.")
    
    n_shared = len(x.intersection(y))
    n_total = len(x.union(y))
    return float("{:.2f}".format(n_shared/n_total))


print(jac(b, c))


In [35]:
a_shingle = set(' '.join([a[i], a[i+1]]) for i in range(len(a)-1))
a_shingle

def two_shingle(x: list):
    x_shingle = set(' '.join([x[i], x[i+1]]) for i in range(len(x)-1))
    return x_shingle

b_shingle = two_shingle(b)
c_shingle = two_shingle(c)

print(jac(b_shingle, c_shingle))

0.12


In [36]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()
d = "green should have smelled more tranquil but somehow it just tasted rotten".split()
e = "joyce enjoyed eating pancakes with ketchup".split()
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled".split()

In [51]:
docs = [a, b, c, d, e, f]

avg_doc_len = float("{:.2f}".format(sum(len(doc) for doc in docs) / len(docs)))   
avg_doc_len

11.67

['his', 'thought', 'process', 'was', 'on']

In [ ]:
import numpy as np

avgdl = sum(len(sentence) for sentence in [a,b,c,d,e,f]) / len(docs)
N = len(docs)

def bm25(word, sentence, k=1.2, b=0.75):
    freq = sentence.count(word)  # or f(q,D) - freq of query in Doc
    tf = (freq * (k + 1)) / (freq + k * (1 - b + b * (len(sentence) / avgdl)))
    N_q = sum([doc.count(word) for doc in docs])  # number of docs that contain the word
    idf = np.log(((N - N_q + 0.5) / (N_q + 0.5)) + 1)
    return round(tf*idf, 4)

In [55]:
a = "purple is the best city in the forest"
b = "there is an art to getting your way and throwing bananas on to the street is not it"  # this is very similar to 'g'
c = "it is not often you find soggy bananas on the street"
d = "green should have smelled more tranquil but somehow it just tasted rotten"
e = "joyce enjoyed eating pancakes with ketchup"
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled"
g = "to get your way you must not bombard the road with yellow fruit"  # this is very similar to 'b'

docs = [a, b, c, d, e, f, g]

In [53]:
from transformers import AutoTokenizer, AutoModel
import torch

/home/peng_luh/__git/search_l3s/search_l3s_search_srv/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [82]:
tokens = tokenizer(docs, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

In [83]:
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [84]:
type(tokens['input_ids'][0])
len(tokens['input_ids'][0])

512

In [85]:
outputs = model(**tokens)

In [86]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [91]:
embeddings = outputs.last_hidden_state

In [92]:
embeddings[0].shape

torch.Size([512, 768])